This notebook has stacking implementation with the following metalearners for **multiclass** classification:
1. Random Forest
2. CatBoost
3. LightGBM

# Imports & Dataset Setup

In [42]:
import numpy as np
seed = 42
np.random.seed(seed)
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, balanced_accuracy_score, classification_report
from sklearn.model_selection import PredefinedSplit, GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform, randint

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
m_mobilenet_train = pd.read_csv("/kaggle/input/fork-of-koa-mobilenetv2/m_mobilenet_train.csv").drop(columns=['Unnamed: 0'])
m_mobilenet_val = pd.read_csv("/kaggle/input/fork-of-koa-mobilenetv2/m_mobilenet_val.csv").drop(columns=['Unnamed: 0'])
m_mobilenet_test = pd.read_csv("/kaggle/input/fork-of-koa-mobilenetv2/m_mobilenet_test.csv").drop(columns=['Unnamed: 0'])

In [4]:
m_densenet_train = pd.read_csv("/kaggle/input/koa-densenet-preds/m_densenet_train.csv").drop(columns=['Unnamed: 0'])
m_densenet_val = pd.read_csv("/kaggle/input/koa-densenet-preds/m_densenet_val.csv").drop(columns=['Unnamed: 0'])
m_densenet_test = pd.read_csv("/kaggle/input/koa-densenet-preds/m_densenet_test.csv").drop(columns=['Unnamed: 0'])

In [5]:
m_yolov8_train = pd.read_csv("/kaggle/input/koa-yolov8-preds/m_yolov8_train.csv").drop(columns=['Unnamed: 0'])
m_yolov8_val = pd.read_csv("/kaggle/input/koa-yolov8-preds/m_yolov8_val.csv").drop(columns=['Unnamed: 0'])
m_yolov8_test = pd.read_csv("/kaggle/input/koa-yolov8-preds/m_yolov8_test.csv").drop(columns=['Unnamed: 0'])

In [6]:
m_yolov8_train.rename(columns = {'FilePath': 'FileName'}, inplace=True)
m_yolov8_val.rename(columns = {'FilePath': 'FileName'}, inplace=True)
m_yolov8_test.rename(columns = {'FilePath': 'FileName'}, inplace=True)

In [7]:
train = m_mobilenet_train.merge(m_yolov8_train).merge(m_densenet_train)
val = m_mobilenet_val.merge(m_yolov8_val).merge(m_densenet_val)
test = m_mobilenet_test.merge(m_yolov8_test).merge(m_densenet_test)

In [8]:
X_train = train[['m_0', 'm_1', 'm_2', 'm_3', 'm_4', 'y_0', 'y_1', 'y_2', 'y_3', 'y_4', 'd_0', 'd_1', 'd_2', 'd_3', 'd_4']]
X_val = val[['m_0', 'm_1', 'm_2', 'm_3', 'm_4', 'y_0', 'y_1', 'y_2', 'y_3', 'y_4', 'd_0', 'd_1', 'd_2', 'd_3', 'd_4']]
X_test = test[['m_0', 'm_1', 'm_2', 'm_3', 'm_4', 'y_0', 'y_1', 'y_2', 'y_3', 'y_4', 'd_0', 'd_1', 'd_2', 'd_3', 'd_4']]
y_train = train[['y_true']]
y_val = val[['y_true']]
y_test = test[['y_true']]

# Random Forest Classifier

In [9]:
rf = RandomForestClassifier(n_jobs = -1)

In [10]:
# For explicitly deefining the validation set in grid search CV
split_index = [-1]*len(X_train) + [0]*len(X_val)
X = np.concatenate((X_train, X_val), axis=0)
y = np.concatenate((y_train, y_val), axis=0).ravel()
pds = PredefinedSplit(test_fold = split_index)

In [11]:
distributions = {
    "n_estimators": [227,228,229],
#     "criterion": ["gini", "entropy", "log_loss"],
    "min_samples_split": [16,17,18]
}

In [12]:
clf = GridSearchCV(estimator = rf,
                   cv=pds,
                   param_grid=distributions, 
                   n_jobs=-1,
                   verbose=1)

In [13]:
clf.fit(X,y)

Fitting 1 folds for each of 9 candidates, totalling 9 fits


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=RandomForestClassifier(n_jobs=-1), n_jobs=-1,
             param_grid={'min_samples_split': [16, 17, 18],
                         'n_estimators': [227, 228, 229]},
             verbose=1)

In [14]:
clf.best_estimator_

RandomForestClassifier(min_samples_split=16, n_estimators=227, n_jobs=-1)

In [16]:
print("Testing on training set:")
print("Accuracy: ", accuracy_score(y_train,clf.best_estimator_.predict(X_train)).round(3))
print("Balanced Accuracy: ", balanced_accuracy_score(y_train,clf.best_estimator_.predict(X_train)).round(3))
print("AUC:", roc_auc_score(y_train,clf.best_estimator_.predict_proba(X_train),multi_class='ovr').round(3))
print("\nTesting on validation set:")
print("Accuracy: ", accuracy_score(y_val,clf.best_estimator_.predict(X_val)).round(3))
print("Balanced Accuracy: ", balanced_accuracy_score(y_val,clf.best_estimator_.predict(X_val)).round(3))
print("AUC:", roc_auc_score(y_val,clf.best_estimator_.predict_proba(X_val),multi_class='ovr').round(3))
print("\nTesting on testing set:")
print("Accuracy: ", accuracy_score(y_test,clf.best_estimator_.predict(X_test)).round(3))
print("Balanced Accuracy: ", balanced_accuracy_score(y_test,clf.best_estimator_.predict(X_test)).round(3))
print("AUC:", roc_auc_score(y_test,clf.best_estimator_.predict_proba(X_test),multi_class='ovr').round(3))

Testing on training set:
Accuracy:  0.942
Balanced Accuracy:  0.948
AUC: 0.997

Testing on validation set:
Accuracy:  0.737
Balanced Accuracy:  0.73
AUC: 0.963

Testing on testing set:
Accuracy:  0.707
Balanced Accuracy:  0.724
AUC: 0.909


# CatBoost

In [17]:
! pip install -q catboost

In [18]:
from catboost import CatBoostClassifier

In [19]:
cbc = CatBoostClassifier(loss_function='MultiClass', verbose=False, random_seed=seed)

In [27]:
distributions = {
    "iterations": [100,120],
    "depth": [10,12],
    "learning_rate": [0.001,0.01,0.1]
}

In [28]:
clf = GridSearchCV(estimator = cbc,
                   cv=pds,
                   param_grid=distributions, 
                   n_jobs=-1,
                   verbose=1)

In [29]:
clf.fit(X,y)

Fitting 1 folds for each of 12 candidates, totalling 12 fits


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=<catboost.core.CatBoostClassifier object at 0x786734817d00>,
             n_jobs=-1,
             param_grid={'depth': [10, 12], 'iterations': [100, 120],
                         'learning_rate': [0.001, 0.01, 0.1]},
             verbose=1)

In [30]:
clf.best_params_

{'depth': 10, 'iterations': 100, 'learning_rate': 0.1}

In [31]:
print("Testing on training set:")
print("Accuracy: ", accuracy_score(y_train,clf.best_estimator_.predict(X_train)).round(3))
print("Balanced Accuracy: ", balanced_accuracy_score(y_train,clf.best_estimator_.predict(X_train)).round(3))
print("AUC:", roc_auc_score(y_train,clf.best_estimator_.predict_proba(X_train),multi_class='ovr').round(3))
print("\nTesting on validation set:")
print("Accuracy: ", accuracy_score(y_val,clf.best_estimator_.predict(X_val)).round(3))
print("Balanced Accuracy: ", balanced_accuracy_score(y_val,clf.best_estimator_.predict(X_val)).round(3))
print("AUC:", roc_auc_score(y_val,clf.best_estimator_.predict_proba(X_val),multi_class='ovr').round(3))
print("\nTesting on testing set:")
print("Accuracy: ", accuracy_score(y_test,clf.best_estimator_.predict(X_test)).round(3))
print("Balanced Accuracy: ", balanced_accuracy_score(y_test,clf.best_estimator_.predict(X_test)).round(3))
print("AUC:", roc_auc_score(y_test,clf.best_estimator_.predict_proba(X_test),multi_class='ovr').round(3))

Testing on training set:
Accuracy:  0.912
Balanced Accuracy:  0.921
AUC: 0.988

Testing on validation set:
Accuracy:  0.701
Balanced Accuracy:  0.699
AUC: 0.917

Testing on testing set:
Accuracy:  0.711
Balanced Accuracy:  0.72
AUC: 0.912


# LightGBM

In [34]:
import lightgbm as lgb

In [35]:
lgbm = lgb.LGBMClassifier(n_jobs=-1,random_state=seed)

In [100]:
distributions = {
    "learning_rate": [0.0089,0.0090,0.0091],
    "n_estimators": [122,123,124],
}

In [101]:
clf = RandomizedSearchCV(estimator = cbc,
                       cv=pds,
                       param_distributions=distributions, 
                       n_jobs=-1,
                       verbose=1,
                       random_state=seed)

In [102]:
clf.fit(X,y)

Fitting 1 folds for each of 9 candidates, totalling 9 fits


RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                   estimator=<catboost.core.CatBoostClassifier object at 0x786734817d00>,
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.0089, 0.009,
                                                          0.0091],
                                        'n_estimators': [122, 123, 124]},
                   random_state=42, verbose=1)

In [103]:
clf.best_params_

{'n_estimators': 123, 'learning_rate': 0.009}

In [104]:
print("Testing on training set:")
print("Accuracy: ", accuracy_score(y_train,clf.best_estimator_.predict(X_train)).round(3))
print("Balanced Accuracy: ", balanced_accuracy_score(y_train,clf.best_estimator_.predict(X_train)).round(3))
print("AUC:", roc_auc_score(y_train,clf.best_estimator_.predict_proba(X_train),multi_class='ovr').round(3))
print("\nTesting on validation set:")
print("Accuracy: ", accuracy_score(y_val,clf.best_estimator_.predict(X_val)).round(3))
print("Balanced Accuracy: ", balanced_accuracy_score(y_val,clf.best_estimator_.predict(X_val)).round(3))
print("AUC:", roc_auc_score(y_val,clf.best_estimator_.predict_proba(X_val),multi_class='ovr').round(3))
print("\nTesting on testing set:")
print("Accuracy: ", accuracy_score(y_test,clf.best_estimator_.predict(X_test)).round(3))
print("Balanced Accuracy: ", balanced_accuracy_score(y_test,clf.best_estimator_.predict(X_test)).round(3))
print("AUC:", roc_auc_score(y_test,clf.best_estimator_.predict_proba(X_test),multi_class='ovr').round(3))

Testing on training set:
Accuracy:  0.863
Balanced Accuracy:  0.879
AUC: 0.973

Testing on validation set:
Accuracy:  0.613
Balanced Accuracy:  0.629
AUC: 0.86

Testing on testing set:
Accuracy:  0.705
Balanced Accuracy:  0.712
AUC: 0.906
